In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import cv2
import csv

In [5]:
tf.keras.backend.set_image_data_format('channels_last')

In [10]:
train_path = './data/train/'
valid_path = './data/valid'
test_path = './data/test'

In [11]:
train_csv = './data/train.csv'
valid_csv = './data/valid.csv'
test_csv = './data/test.csv'

In [12]:
def append_ext(fn):    # Helper function to append extention name for image id
    return fn + ".jpg"

In [13]:
# Data frames for image names and labels
train_df = pd.read_csv(train_csv)
valid_df = pd.read_csv(valid_csv)
test_df = pd.read_csv(test_csv)

In [14]:
train_df["image_id"] = train_df["image_id"].apply(append_ext)

In [15]:
valid_df["image_id"] = valid_df["image_id"].apply(append_ext)

In [16]:
test_df["image_id"] = test_df["image_id"].apply(append_ext)

In [17]:
train_df[:3]

,image_id,breed
0,597e09c5d0.jpg,agkistrodon-contortrix
1,a14a5ace05.jpg,pituophis-catenifer
2,5a35436f64.jpg,thamnophis-proximus


In [18]:
IMG_DIM=224
bs = 64

In [19]:
datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

In [20]:
train_batches = datagen.flow_from_dataframe(
    train_df,
    directory=train_path,
    target_size=(IMG_DIM,IMG_DIM),
    x_col="image_id",
    y_col="breed",
    batch_size=bs
)

Found 4958 validated image filenames belonging to 35 classes.


In [21]:
valid_batches = datagen.flow_from_dataframe(
    valid_df,
    directory=valid_path,
    target_size=(IMG_DIM, IMG_DIM),
    x_col="image_id",
    y_col="breed",
    batch_size=bs
)

Found 550 validated image filenames belonging to 35 classes.


In [22]:
ep=20

In [23]:
NAME="CUSTOM-MODEL-I"
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.24))
model.add(Dense(35,activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 52, 52, 128)       36992     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 86528)            

In [25]:

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])
# Fit
model.fit(
    x=train_batches,
    steps_per_epoch= train_batches.n // train_batches.batch_size,
    validation_data= valid_batches,
    validation_steps= valid_batches.n // valid_batches.batch_size,
    epochs=ep
)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
77/77 [==============================] - ETA: 0s - loss: 6.6968 - accuracy: 0.0691WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001E0F39FD9D8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) 

In [26]:
model.save(NAME)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: CUSTOM-MODEL-I\assets
